### Boilerplate code - llm initiation

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0, 
    model="gemini-2.5-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)

google_llm.invoke("Hello")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bccb1-b414-7050-9e5e-46f60d6d9d89-0', usage_metadata={'input_tokens': 2, 'output_tokens': 261, 'total_tokens': 263, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 252}})

## Message state

In [3]:
from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content="Hi, there!"), 
            HumanMessage(content="Hello AI!"), 
            AIMessage(content="How can i help you?"), 
            HumanMessage(content="Why are you angry at me?")
        ]

google_llm.invoke(messages)

AIMessage(content='Oh, I\'m not angry at all! I apologize if anything I said came across that way. As an AI, I don\'t actually experience emotions like anger. My previous message, "How can I help you?", was just a standard way for me to offer assistance and see what you\'d like to talk about or achieve.\n\nPlease tell me, what makes you feel like I might be angry? I want to make sure I communicate clearly and helpfully.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bccb3-ee26-7210-ad39-a174528993a2-0', usage_metadata={'input_tokens': 24, 'output_tokens': 263, 'total_tokens': 287, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 167}})

In [ ]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class MessageState(TypedDict):
    messages: list[AnyMessage] # AIMessage(Role is assistant), HumanMessage(Role is human automatically)

class MessageState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


# Default Behavior (Overwrite)
# state = {"messages": ["Hello"]}
# new_update = {"messages": ["Hi there"]}
# state.update(new_update)

### Intro on add_messages

In [ ]:
from langgraph.graph.message import add_messages

data1 = [{"role": "human", "content": "test"}] # role and content are must keys
data2 = [{"role": "ai", "content": "test2"}]

added_data = add_messages(data1, data2)

added_data # similar operation like .extend

### Langgraph - Building chatbot with MessageState

In [ ]:
from langgraph.graph import MessagesState, StateGraph, START, END
from IPython.display import display, Image
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage
from langgraph.graph.message import add_messages
from typing import Annotated

class MessagesState(MessagesState):
    pass

# class MessagesState(TypedDict):
#     messages: Annotated[list[AnyMessage], add_messages]

def chatbot(state: MessagesState):
    return {"messages": google_llm.invoke(state.get('messages'))}

In [ ]:

builder = StateGraph(MessagesState)
builder.add_node("chatbot", chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile()

In [ ]:

graph.invoke({"messages": HumanMessage(content="Hello")}) # HumanMessage can be list of direct as internally it uses add_messages reducer i.e .extend so final will be list anyways
graph.invoke({"messages": HumanMessage(content="What is your name?")})
graph.invoke({"messages": HumanMessage(content="I want to see you")}) # returns final state of graph execution

# display(Image(graph.get_graph().draw_mermaid_png()))

### ====== Rough Notes ====== ###

##### .pretty_print()

In [6]:
from langchain_core.messages import AIMessage, HumanMessage

print(HumanMessage(content=f"Yes, that's right.",name="Lance").pretty_print())

================================ Human Message =================================
Name: Lance

Yes, that's right.
None


##### .bind_tools just gives the tools and args. we have to manually run the function
##### in simple terms, tool function has to be called by us and should append the result to messages

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, ToolMessage

# 1. Define Tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

llm = ChatOpenAI()
llm_with_tools = llm.bind_tools([multiply])

# 2. First Call: LLM decides to use the tool
query = "What is 5 multiplied by 3?"
messages = [HumanMessage(content=query)]
ai_msg = llm_with_tools.invoke(messages)

# Check what the LLM returned
print(f"LLM Wants: {ai_msg.tool_calls}")
# Output: [{'name': 'multiply', 'args': {'a': 5, 'b': 3}, 'id': '...'}]

# 3. Manual Execution: We run the function ourselves
tool_call = ai_msg.tool_calls[0]
if tool_call["name"] == "multiply":
    result = multiply(**tool_call["args"]) # We run python code here

# 4. Feed result back to LLM
# We must append the AI's request AND our Tool result
messages.append(ai_msg)
messages.append(ToolMessage(content=str(result), tool_call_id=tool_call["id"]))

# 5. Second Call: LLM sees the result and answers
final_response = llm_with_tools.invoke(messages)
print(f"Final Answer: {final_response.content}")
# Output: "5 multiplied by 3 is 15."

##### create_react_agent from langraph (needs only mandatory two arguments)

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

# Define the System Prompt here
prompt = "You are a pirate. Answer everything with 'Arrr' and pirate slang."

agent = create_react_agent(
    model=model, 
    tools=[], 
    state_modifier=prompt  # <--- HERE, optional
)

res = agent.invoke({"messages": [("human", "Hello")]})
print(res["messages"][-1].content)
# Output: "Arrr matey! What brings ye to me ship?"

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.graph.message import add_messages
from langgraph.graph import MessagesState, StateGraph, START, END